# Cell type Means
## 11-11-24 use Nov 2024 version 2d object and Ensembl ID TF's.
## This code generates the mean log normal values for each cell type
### Trimmed version with mean only, does not have percent of cells.





## General Import Functions

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import seaborn as sns
import anndata
import string
import gc
from anndata import read_h5ad
from anndata import read_csv
from pandas import DataFrame
import h5py



In [2]:
#data switches

ALL_GENES = False

DataSet = 'Full'  # means all tissues
TenXOnly = True   # don't use plate data
Raw = False       # use raw counts, not log normalized data

In [16]:
#Stuff coming in
SHARED_DATA_DIR = '/hpc/projects/tabula_sapiens/singlecell_transcriptomics/TSP1_30_Paper2_Version2d_Oct2024/full_object/'

#TF_DATA = 'HumanTranscriptionFactors.csv'
TF_DATA_ENSEMBL = 'HumanTranscriptionFactorsEnsembl.csv'
META_DATA = 'TSP1_30_metadata_min200_2500_decontx_scvi_donorassay_version2d_20241112.csv'


# data in   (full means all tissues)
if DataSet == 'Full':
    DATA = 'TSP1_30_min200_2500_decontx_scvi_donorassay_version2d_20241126.h5ad' 

In [7]:
print('Directory with data in = ', SHARED_DATA_DIR)
print()
print('Data In = ', DATA)
print()
print('MetaData In = ', META_DATA)
print()

Directory with data in =  /hpc/projects/tabula_sapiens/singlecell_transcriptomics/TSP1_30_Paper2_Version2d_Oct2024/full_object/

Data In =  TSP1_30_min200_2500_decontx_scvi_donorassay_version2d_20241126.h5ad

MetaData In =  TSP1_30_metadata_min200_2500_decontx_scvi_donorassay_version2d_20241112.csv



# Read object and list of transcription factors

In [8]:
print(SHARED_DATA_DIR + DATA)
#adata = read_h5ad(SHARED_DATA_DIR + DATA)
adata


/hpc/projects/tabula_sapiens/singlecell_transcriptomics/TSP1_30_Paper2_Version2d_Oct2024/full_object/TSP1_30_min200_2500_decontx_scvi_donorassay_version2d_20241126.h5ad


AnnData object with n_obs × n_vars = 1136218 × 61806
    obs: 'donor', 'tissue', 'anatomical_position', 'method', 'cdna_plate', 'library_plate', 'notes', 'cdna_well', 'old_index', 'assay', 'sample_id', 'replicate', '10X_run', '10X_barcode', 'ambient_removal', 'donor_method', 'donor_assay', 'donor_tissue', 'donor_tissue_assay', 'cell_ontology_class', 'cell_ontology_id', 'compartment', 'broad_cell_class', 'free_annotation', 'manually_annotated', 'published_2022', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ercc', 'pct_counts_ercc', '_scvi_batch', '_scvi_labels', 'scvi_leiden_donorassay_full', 'age', 'sex', 'ethnicity', 'sample_number'
    var: 'ensembl_id', 'gene_symbol', 'genome', 'mt', 'ercc', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'mean', 'std'
    uns: '_scvi_manager_uuid', '_scvi_uuid', '_training_mode', 'age_colors', 'assay_colors', 'compartment_colors', 'donor_colors', 'leiden', 'log1p', 'method_color

In [9]:
if TenXOnly:
    adata = adata[adata.obs.method == '10X']

In [17]:
def mapTFsbyEnsemblIDlist():
    TF_data = pd.read_csv(TF_DATA_ENSEMBL)

    adata_table = pd.DataFrame()
    adata_table['ensembl_id'] = adata.var.ensembl_id.str.split('.', expand=True)[0]
    adata_table['gene_symbol'] = adata.var.gene_symbol
    
    # slice to just TF's
    TF_adata_table = adata_table[adata_table.ensembl_id.isin(TF_data.TF_names)]
    TF_adata_table.reset_index()
    TF_adata_table.sort_values('ensembl_id')
    
    return(TF_adata_table)

In [18]:
mapTFsbyEnsemblIDlist()

,ensembl_id,gene_symbol
index,,
NFYA,ENSG00000001167,NFYA
ARX,ENSG00000004848,ARX
HOXA11,ENSG00000005073,HOXA11
MEOX1,ENSG00000005102,MEOX1
SOX8,ENSG00000005513,SOX8
...,...,...
PCGF2,ENSG00000277258,PCGF2
ZNF670,ENSG00000277462,ZNF670
ZNF8,ENSG00000278129,ZNF8


In [19]:
if ALL_GENES:
    TF_list =  list(adata.var.index)
else:   #just the TF's
    TF_list = mapTFsbyEnsemblIDlist().index
len(TF_list)

1637

## Make TF x Cell Type dataframe of mean expression

In [20]:
CellTypesList = list(adata.obs.cell_ontology_class.unique())
#CellTypesList = ['basal cell','keratinocyte']  #for debug
len(CellTypesList)

175

In [21]:
TF_df = DataFrame()

for CT in CellTypesList:
    CellsOfThisType = list(adata.obs[adata.obs.cell_ontology_class == CT].index)

    CT_adata = adata[CellsOfThisType,TF_list]
    if Raw:
        TF_means = CT_adata.layers['raw_counts'].mean(0)
    else:
        TF_means = CT_adata.layers['log_normalized'].mean(0)
        
    df = pd.DataFrame(TF_means, columns = TF_list, index = [CT])

    TF_df = pd.concat([TF_df,df])

TF_df.T

,macrophage,mesenchymal stem cell of adipose tissue,mast cell,neutrophil,fibroblast,"cd8-positive, alpha-beta t cell",monocyte,"cd4-positive, alpha-beta t cell",pericyte,endothelial cell,...,regular atrial cardiac myocyte,neuron,oocyte,taste receptor cell,serous cell of epithelium of bronchus,kidney epithelial cell,retinal pigment epithelial cell,keratinocyte,enterocyte of epithelium proper of jejunum,leukocyte
index,,,,,,,,,,,,,,,,,,,,,
NFYA,0.154053,0.052249,0.086978,0.279022,0.079945,0.133177,0.124976,0.105786,0.131542,0.157092,...,0.057238,0.051793,0.000000,0.100900,0.000000,0.037919,0.019420,0.056283,0.168485,0.182553
ARX,0.000519,0.000000,0.000000,0.000018,0.000084,0.000219,0.000308,0.000132,0.000230,0.000891,...,0.000000,0.000000,0.244119,0.000000,0.000000,0.001467,0.000000,0.000000,0.000000,0.000000
HOXA11,0.000398,0.019499,0.000475,0.000058,0.063043,0.000069,0.000350,0.000036,0.012761,0.000850,...,0.000000,0.000000,0.000000,0.023355,0.000000,0.000461,0.000000,0.000000,0.000000,0.000000
MEOX1,0.002769,0.004737,0.002189,0.000715,0.005107,0.003749,0.004584,0.010688,0.016106,0.231181,...,0.003055,0.017873,0.000000,0.036137,0.000000,0.001310,0.000000,0.000000,0.000000,0.000000
SOX8,0.001229,0.020782,0.002324,0.000033,0.010603,0.002740,0.000562,0.007560,0.053537,0.008764,...,0.008482,0.093364,0.000000,0.000000,0.155411,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PCGF2,0.019405,0.116857,0.005018,0.000536,0.127053,0.002726,0.006945,0.002373,0.111260,0.273615,...,0.264298,0.092275,0.000000,0.086373,0.063571,0.120260,0.204707,0.000000,0.023220,0.000000
ZNF670,0.010195,0.009335,0.017056,0.000586,0.018906,0.006841,0.007379,0.006090,0.013424,0.030096,...,0.015231,0.013832,0.116221,0.000000,0.000000,0.016763,0.000000,0.079990,0.009989,0.000000
ZNF8,0.020019,0.019603,0.012939,0.006057,0.021896,0.018999,0.018273,0.019217,0.031671,0.034838,...,0.046540,0.007992,0.000000,0.000000,0.026132,0.037138,0.000000,0.000000,0.014616,0.000000


In [35]:
#convert cell type names to indexes without ' ' and '-'
TF_rownames = list(TF_df.index)
TF_rownames = [sub.replace(' ', '_') for sub in TF_rownames]
TF_rownames = [sub.replace('-', '_') for sub in TF_rownames]
TF_rownames = [sub.replace(',', '_') for sub in TF_rownames]
TF_df.index = TF_rownames
TF_df


index,NFYA,ARX,HOXA11,MEOX1,SOX8,ZNF195,ZFX,YBX2,ZNF263,DLX6,...,SEBOX,ZNF280B,ZNF2,HNF1B,DACH1,PCGF2,ZNF670,ZNF8,ZNF229,NR2E3
macrophage,0.154053,0.000519,0.000398,0.002769,0.001229,0.083150,0.421626,0.000253,0.114945,0.000113,...,0.000000,0.009422,0.015929,0.001249,0.063781,0.019405,0.010195,0.020019,0.002689,0.001045
mesenchymal_stem_cell_of_adipose_tissue,0.052249,0.000000,0.019499,0.004737,0.020782,0.088510,0.255203,0.000088,0.092353,0.000391,...,0.000000,0.004263,0.015310,0.000074,0.024646,0.116857,0.009335,0.019603,0.004925,0.005257
mast_cell,0.086978,0.000000,0.000475,0.002189,0.002324,0.069971,0.276791,0.000142,0.056720,0.000000,...,0.000000,0.004928,0.012086,0.001421,0.377390,0.005018,0.017056,0.012939,0.002050,0.001533
neutrophil,0.279022,0.000018,0.000058,0.000715,0.000033,0.009683,0.354291,0.000035,0.209102,0.000018,...,0.000000,0.000845,0.002228,0.000495,1.051913,0.000536,0.000586,0.006057,0.000084,0.000296
fibroblast,0.079945,0.000084,0.063043,0.005107,0.010603,0.092513,0.305828,0.000116,0.084879,0.002213,...,0.000023,0.002570,0.020048,0.001000,0.111145,0.127053,0.018906,0.021896,0.002380,0.005355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
kidney_epithelial_cell,0.037919,0.001467,0.000461,0.001310,0.000000,0.191532,0.239228,0.005082,0.056315,0.000232,...,0.000070,0.042522,0.016841,0.471793,0.096221,0.120260,0.016763,0.037138,0.013244,0.001868
retinal_pigment_epithelial_cell,0.019420,0.000000,0.000000,0.000000,0.000000,0.051691,0.192917,0.016006,0.066641,0.000000,...,0.000000,0.058852,0.039459,0.000000,0.019574,0.204707,0.000000,0.000000,0.020606,0.019311
keratinocyte,0.056283,0.000000,0.000000,0.000000,0.000000,0.111229,0.129412,0.000000,0.079990,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.079990,0.000000,0.000000,0.000000
enterocyte_of_epithelium_proper_of_jejunum,0.168485,0.000000,0.000000,0.000000,0.000000,0.180533,0.454409,0.109400,0.111378,0.000000,...,0.000000,0.006153,0.018718,0.190655,0.879147,0.023220,0.009989,0.014616,0.000000,0.008909


In [36]:
# read metatdata that goes with the full objects
# modify cell types names to ensure they  match
metadata = pd.read_csv(SHARED_DATA_DIR+META_DATA)
metadata = metadata[metadata.method == '10X']
TF_rownames = list(metadata.cell_ontology_class)
TF_rownames = [sub.replace(' ', '_') for sub in TF_rownames]
TF_rownames = [sub.replace('-', '_') for sub in TF_rownames]
TF_rownames = [sub.replace(',', '_') for sub in TF_rownames]
metadata.cell_ontology_class = TF_rownames


/tmp/ipykernel_27552/1238109005.py:3: DtypeWarning: Columns (5,6,7,8,9,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(SHARED_DATA_DIR+META_DATA)


In [37]:
# check they are the same 
metaset = set(metadata.cell_ontology_class.unique())
TFdfset = set(TF_df.index)
print(TFdfset.difference(metaset))
print(metaset.difference(TFdfset))

set()
set()


In [38]:
TF_df.to_csv(DATA_OUT_DIR+DATA_CT_TABLE)
metadata.to_csv(DATA_OUT_DIR+META_DATA_OUT)

print('TFxCT saved as ', DATA_OUT_DIR+DATA_CT_TABLE )
print('Meta data clean saved as ', DATA_OUT_DIR+META_DATA_OUT )

TFxCT saved as  /oak/stanford/groups/quake/jonesbob/TF_TS_Analysis/TFonly/TSP1_30_by_CellType_Full_Version2d_Nov2024_LogNorm_10XOnly_Mean_TFonly.csv
Meta data clean saved as  /oak/stanford/groups/quake/jonesbob/TF_TS_Analysis/TFonly/TSP1_30_version2d_10X_SS_2024_11_12_metadata_CLEAN_TFonly.csv


In [39]:
##################################################################################

In [40]:
len(list(metadata.cell_ontology_class.unique()))

175

In [41]:
# legacy switches.  Not used

#data out
if DataSet == 'Full':
    DATA_TF_ = 'TSP1_30_TF_by_CellType_Full_version2c_202409234'

if Raw:
    DATA_TF_ = DATA_TF_+'_Raw'
else:
    DATA_TF_ = DATA_TF_+'_LogNorm'

if TenXOnly:
    DATA_TF_ = DATA_TF_+'_10XOnly'
else:
    DATA_TF_ = DATA_TF_+'_10XnSS2'    
    
DATA_TF_TABLE = DATA_TF_+'.csv'

print('Directory with data in = ', SHARED_DATA_DIR)
print('Data In = ', DATA)

print()
print('Directory for data out = ', DATA_OUT_DIR)
print('Data Out = ', DATA_CT_TABLE)

Directory with data in =  /oak/stanford/groups/quake/shared/TabulaSapiensTSP1_30_Paper2_Version2d_Nov2024/
Data In =  full_object/TSP1_30_min200_2500_decontx_scvi_donorassay_version2d_20241112.h5ad

Directory for data out =  /oak/stanford/groups/quake/jonesbob/TF_TS_Analysis/
Data Out =  TFonly/TSP1_30_by_CellType_Full_Version2d_Nov2024_LogNorm_10XOnly_Mean_TFonly.csv


In [28]:
adata.X[0:100, 0:100].data

array([0.81211257, 1.254283  , 0.81211257, ..., 1.021765  , 1.8459764 ,
       1.021765  ], dtype=float32)

In [30]:
adata

View of AnnData object with n_obs × n_vars = 1093048 × 61806
    obs: 'donor', 'tissue', 'anatomical_position', 'method', 'cdna_plate', 'library_plate', 'notes', 'cdna_well', 'old_index', 'assay', 'sample_id', 'replicate', '10X_run', '10X_barcode', 'ambient_removal', 'donor_method', 'donor_assay', 'donor_tissue', 'donor_tissue_assay', 'cell_ontology_class', 'cell_ontology_id', 'compartment', 'broad_cell_class', 'free_annotation', 'manually_annotated', 'published_2022', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ercc', 'pct_counts_ercc', '_scvi_batch', '_scvi_labels', 'scvi_leiden_donorassay_full', 'age', 'sex', 'ethnicity', 'sample_number'
    var: 'ensembl_id', 'gene_symbol', 'genome', 'mt', 'ercc', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'mean', 'std'
    uns: '_scvi_manager_uuid', '_scvi_uuid', '_training_mode', 'age_colors', 'assay_colors', 'compartment_colors', 'donor_colors', 'leiden', 'log1p', 'meth

In [33]:
adata.layers['raw_counts']

<1093048x61806 sparse matrix of type '<class 'numpy.int32'>'
	with 3542059822 stored elements in Compressed Sparse Row format>

In [ ]:
df_rc = adata.to_df(layer='raw_counts')
df_rc

In [1]:
adata

NameError: name 'adata' is not defined